<a href="https://colab.research.google.com/github/niou12/ConsMedInt/blob/main/DiagnosticMedicalIntelligent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importation de toutes les librairies necessaires
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tkinter import *
import numpy as np
import pandas as pd
import os

ceci est un projet de consultaion medicale intelligent pour faire un diagnostic de la santé d'un patient. On prend ici l'ophtalmologie qui est la maladie de l'oeil.

In [ ]:
#liste des symptomes de la maladie de l'oeil (l1)
l1 = ['Vision floue','Éblouissement','Vision nocturne réduite','Vision en double','Vision centrale dégradée','Distorsion des lignes droites','Tâche sombre dans le champ visuel','vision périphérique réduite','Maux de tête','Douleur oculaire','Sensibilité à la lumière','Sécrétion oculaire']


In [ ]:
#liste des catégories de la maladie ophtalmologie
maladie=['cataracte','dmla','glaucome','keratite']

#maladie = [df['prediction'].unique()]
#print(maladie)


In [ ]:
l2=[]
for i in range(0,len(l1)):
    l2.append(0)
print(l2)

In [17]:
#IMPORTER MES DONNEES DEPUIS
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import codecs

# Chemin du fichier CSV d'origine et du fichier de sortie UTF-8
chemin_entree = "/content/drive/MyDrive/Colab Notebooks/prototp.csv"
chemin_sortie = "/content/drive/MyDrive/Colab Notebooks/prototp_utf8.csv"

# Ouvrir le fichier d'entrée avec l'encodage d'origine
with codecs.open(chemin_entree, 'r', encoding='ISO-8859-1') as file:
    contenu = file.read()

# Écrire le contenu dans un nouveau fichier avec l'encodage UTF-8
with codecs.open(chemin_sortie, 'w', encoding='utf-8') as file:
    file.write(contenu)


In [20]:
#IMPORTER UN FICHIER XL CONTENANT LA LISTE DES SYMPTOMES GROUPES PAR MALADIE CORRESPONDANTE
#grouper les symptomes par maladie


df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/prototp.csv")
DF=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/prototp_utf8.csv', index_col='prediction')
#DF= pd.read_csv('training.csv', index_col='prognosis')

#REMPLACER LES VALEURS DU FICHIER IMPORTE PAR PANDAS EN UTILISANT SES FONCTIONS
df.replace({'prediction':{'Vision floue':0,'Éblouissement':1,'Vision nocturne réduite':2,'Vision en double':3,'Vision centrale dégradée':4,
                          'Distorsion des lignes droites':5,'Tâche sombre dans le champ visuel':6,'vision périphérique réduite':7,
                          'Maux de tête':8,'Douleur oculaire':9,'Sensibilité à la lumière':10,'Sécrétion oculaire':11}},inplace=True)
DF.head()

,vision_floue,eblouissement,vision_nocturne_reduite,vision_en_double,vision_centrale_degradee,distorsion_ligne_droite,tache_sombre,vision_preipherique_reduite,maux_de_tete,douleur_oculaire,secretion_oculaire,sensible_lumiere
prediction,,,,,,,,,,,,
1,1,1,1,0,0,0,0,0,0,0,0,cataracte
0,0,0,0,1,1,1,0,0,0,0,0,dmla
1,0,0,0,0,0,0,1,1,0,0,0,glaucome
1,0,0,0,0,0,0,0,0,1,1,1,keratite


In [ ]:


#REPRESENTER LES DONNEES PAR DES GRAPHES
def plotPerColumnDistribution(df1, nGraphshown, nGraphPerRow):
	nunique = df1.nunique()
	df1 = [col for col in df if nuninque[col] > 1 and nunique[col]] < 5
	nRow, nCol = df1.shape
	columnNames = list(df1)
	nGraphRow = (nCol + nGraphRow - 1) / nGraphhRow
	plt.figure(num = Non, figsize = (6 * nGraphRow, 8 * nGraphRow), dpi = 8, facecolor = 'w', edgecolor = 'k')
	for i in range(min(nGraphRow, nGraphShown)):
		plt.subplot(nGraphRow, nGraphPerRow, i + 1)
		columnDf = df.iloc[:, i]
		if(not np.issubdtype(type(columnDf.iloc[0]), np.number)):
			valueCounts = columnDf.value_counts()
			valueCounts.plot.bar()
		else:
			columnDf.hist()
		plt.ylabel('counts')
		plt.xticks(rotation = 90)
		plt.title(f'{columnNames[i]} (column {i})')
	plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
	plt.show()